In [ ]:
!pip install langchain
!pip install jq
!pip install sentence-transformers

In [1]:
from huggingface_hub import hf_hub_download, snapshot_download
from langchain_community.document_loaders import JSONLoader
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm
import json
import pickle
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


In [ ]:
snapshot_download(repo_id="JetBrains-Research/commit-chronicle", repo_type="dataset", cache_dir='./')

In [11]:

folder_path = './datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

df = pq.read_table(folder_path + '/train-00000-of-00061-2a7ccc8e843f5f5b.parquet').to_pandas()

In [31]:


# Create empty DataFrames for each language
languages = ['Java', 'Python', 'C#', 'C++', 'JavaScript']
dfs = {lang: [] for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=100000, desc=lang) for lang in languages}

folder_path = './datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

files = glob.glob(os.path.join(folder_path, 'train*'))
for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages:
        # Filter rows where language column matches the current language
        lang_df = df[(df['language'] == lang) & (df['mods'].apply(len) == 1) & (df['mods'].apply(lambda x: x[0]['change_type']) == 'MODIFY') & (df['mods'].apply(lambda x: len(f"diff --git a/{x[0]['old_path']} b/{x[0]['new_path']} {x[0]['diff']}") <= 3000))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            diff = row['mods'][0]
            old_path = 'a/' + diff['old_path']
            new_path = 'b/' + diff['new_path']
            diff_content = diff['diff']  # assume diff_content is an empty string
            item = {
                'msg': row['message'],
                'diff': f"diff --git {old_path} {new_path} {diff_content}",
                'lang': lang
            }
            dfs[lang].append(item)
            bars[lang].update(1)

            # Check if the language has reached 100,000 rows
            if len(dfs[lang]) >= 100000:
                print(f"Reached 100,000 rows for {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break
    # Break out of the loop if all languages have reached 100,000 rows
    if not languages:
        break

Java:   0%|          | 0/100000 [00:00<?, ?it/s]

Python:   0%|          | 0/100000 [00:00<?, ?it/s]

C#:   0%|          | 0/100000 [00:00<?, ?it/s]

C++:   0%|          | 0/100000 [00:00<?, ?it/s]

JavaScript:   0%|          | 0/100000 [00:00<?, ?it/s]

Reached 100,000 rows for Python
Reached 100,000 rows for JavaScript
Reached 100,000 rows for Java
Reached 100,000 rows for C++
Reached 100,000 rows for C#


In [34]:
# Create a single list of all items
all_items = [item for lang in dfs for item in dfs[lang]]

In [36]:

# Dump the data to a JSON file
with open('../data/chronicle/chronicle_rag_db.json', 'w') as f:
    json.dump(all_items, f, indent=4)

In [42]:
data = {}
for index, item in enumerate(all_items):
    data[index+1] = item['msg']

with open("../data/chronicle/rag_msg.pkl", "wb") as f:
    pickle.dump(data, f)

In [5]:
with open("../data/chronicle/rag_msg.pkl", "rb") as f:
    msg_data = pickle.load(f)

In [4]:
diff_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_rag_db.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

In [17]:

language_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_rag_db.json',
    jq_schema='.[].lang',
    text_content=False)

language_data = language_loader.load()

In [5]:
language_data[0]

Document(page_content='Java', metadata={'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\chronicle\\chronicle_rag_db.json', 'seq_num': 1})

In [30]:
# 1-1000 data use java splitter, 1000-2000 use cpp splitter, 2000-3000 use csharp splitter, 3000-4000 use python splitter, 4000-5000 use javascript splitter


# languages = [Language.JAVA, Language.CPP, Language.CSHARP, Language.PYTHON, Language.JS]
languages = [Language.PYTHON, Language.JS]
splitters = [RecursiveCharacterTextSplitter.from_language(language, chunk_size=300, chunk_overlap=60) for language in languages]


# language_dict = {'java': 0, 'c++': 1, 'c#': 2, 'python': 3, 'javascript': 4}
language_dict = {'Python': 0, 'JavaScript': 1}

In [47]:
diff_split = []
for i, doc in tqdm(enumerate(diff_data), total=len(diff_data), desc="Processing documents"):
    if language_data[i].page_content in language_dict:
        diff_split.append(doc)
        # diff_split += splitters[language_dict[language_data[i].page_content]].split_documents([doc])

Processing documents:   0%|          | 0/500000 [00:00<?, ?it/s]

In [48]:
type(diff_split[0])

langchain_core.documents.base.Document

In [2]:
# Define the path to the pre-trained model you want to use
modelPath = "microsoft/codebert-base"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    cache_folder = '../models',
    
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with MEAN pooling.


In [49]:
# Define the path to the pre-trained model you want to use
modelPath = "mixedbread-ai/mxbai-embed-large-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    cache_folder = '../models',
    
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [3]:
db = Chroma(persist_directory="./chroma_chronicle_db_mxbai_nochunk", embedding_function=embeddings)

In [15]:
db_2 = Chroma(persist_directory="./chroma_chronicle_db_mxbai_300chunk", embedding_function=embeddings)

In [50]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(diff_split, embeddings, persist_directory="./chroma_chronicle_db_mxbai_nochunk")

In [ ]:
db.persist()

In [6]:
test_diff_loader = JSONLoader(
    file_path='../data/chronicle/rag_dev/chronicle_test_data_dev.json',
    jq_schema='.[].diff',
    text_content=False)

test_diff_data = test_diff_loader.load()

In [11]:

with open('../data/chronicle/rag_dev/chronicle_test_data_dev.json', 'r') as f:
    test_data = json.load(f)

In [10]:
len(test_data)

2000

In [30]:
def similarity_search(documents):
    # Initialize an empty dictionary to store aggregate scores for each candidate ID
    aggregate_scores = {}

    # Iterate through each document in the documents list
    for document in documents:
        # Apply similarity search function to the document
        results = db.similarity_search_with_relevance_scores(document.page_content, score_threshold=0.0)
        # Iterate through the results for each document
        for candidate_doc in results:
            id = candidate_doc[0].metadata['seq_num']
            score = candidate_doc[1]
            # Update the aggregate score for the candidate ID
            aggregate_scores[id] = aggregate_scores.get(id, 0) + score

    # Find the candidate ID with the highest aggregate score
    if aggregate_scores:
        max_candidate_id = max(aggregate_scores, key=aggregate_scores.get)
    else:
        max_candidate_id = -1

    return max_candidate_id

In [35]:
documents = splitters[language_dict[test_data[0]['lang']]].split_documents([test_diff_data[0]])

In [52]:
test_data[0]['diff']

'diff --git a/fkie_node_manager/src/fkie_node_manager/editor/editor.py b/fkie_node_manager/src/fkie_node_manager/editor/editor.py @@ -168,6 +168,7 @@ class Editor(QMainWindow):\nfor f in filenames:\nif f:\nself.on_load_request(f, search_text, only_launch=True)\n+ self.log_dock.setVisible(False)\n# def __del__(self):\n# print "******** destroy", self.objectName()\n'

In [32]:
documents[0].page_content

'diff --git a/fkie_node_manager/src/fkie_node_manager/editor/editor.py b/fkie_node_manager/src/fkie_node_manager/editor/editor.py @@ -168,6 +168,7 @@ class Editor(QMainWindow):\nfor f in filenames:\nif f:\nself.on_load_request(f, search_text, only_launch=True)\n+ self.log_dock.setVisible(False)'

In [33]:
db_2.similarity_search_with_relevance_scores(documents[0].page_content)

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='diff --git a/qualcoder/manage_files.py b/qualcoder/manage_files.py @@ -102,6 +102,7 @@ class DialogManageFiles(QtWidgets.QDialog):\ndefault_import_directory = os.path.expanduser("~")\nattribute_names = [] # list of dictionary name:value for AddAtributewww.git dialog\nparent_textEdit = None\n+ dialog_list = []\ndef __init__(self, app, parent_textEdit):\n@@ -110,6 +111,7 @@ class DialogManageFiles(QtWidgets.QDialog):\nself.default_import_directory = self.app.settings[\'directory\']\nself.parent_textEdit = parent_textEdit\nself.attributes = []\n+ self.dialog_list = []\nQtWidgets.QDialog.__init__(self)\nself.ui = Ui_Dialog_manage_files()\nself.ui.setupUi(self)\n@@ -509,13 +511,14 @@ class DialogManageFiles(QtWidgets.QDialog):\nif restricted:\ntitle += "RESTRICTED EDIT"\nself.text_view.setWindowTitle(title)\n- ok 

[(Document(page_content='diff --git a/qualcoder/manage_files.py b/qualcoder/manage_files.py @@ -102,6 +102,7 @@ class DialogManageFiles(QtWidgets.QDialog):\ndefault_import_directory = os.path.expanduser("~")\nattribute_names = [] # list of dictionary name:value for AddAtributewww.git dialog\nparent_textEdit = None\n+ dialog_list = []\ndef __init__(self, app, parent_textEdit):\n@@ -110,6 +111,7 @@ class DialogManageFiles(QtWidgets.QDialog):\nself.default_import_directory = self.app.settings[\'directory\']\nself.parent_textEdit = parent_textEdit\nself.attributes = []\n+ self.dialog_list = []\nQtWidgets.QDialog.__init__(self)\nself.ui = Ui_Dialog_manage_files()\nself.ui.setupUi(self)\n@@ -509,13 +511,14 @@ class DialogManageFiles(QtWidgets.QDialog):\nif restricted:\ntitle += "RESTRICTED EDIT"\nself.text_view.setWindowTitle(title)\n- ok = self.text_view.exec_()\n- if not ok:\n- return\n+ if self.dialog_list != []:\n+ self.dialog_list = []\n+ self.dialog_list.append(self.text_view)\n+ self.

In [53]:
db.similarity_search_with_relevance_scores(test_data[0]['diff'])

[(Document(page_content='diff --git a/application.py b/application.py @@ -138,10 +138,10 @@ class EditRoot(np.PropertyOwner):\nreturn class_names\n- def tree_remove(self):\n+ def recursive_remove(self):\n# clear all\nfor n in self.children:\n- n.tree_remove()\n+ n.recursive_remove()\n#self.root.children = None\ndef find_widget_from_path(self, path):\n@@ -207,6 +207,7 @@ class Application(EditRoot):\nelif sys.platform=="darwin":\n_PROPERTY_HELP["output_path"] = "Output file or directory; double click label to show in Finder"\n+ WX_CLASS = "wxApp"\nIS_WINDOW = IS_SIZER = IS_TOPLEVEL = IS_SLOT = False\nCHILDREN = None # any number of toplevel windows etc.\n', metadata={'seq_num': 145023, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\chronicle\\chronicle_rag_db.json'}),
  -62.74814282497707),
 (Document(page_content='diff --git a/qualcoder/manage_files.py b/qualcoder/manage_files.py @@ -90,11 +90,10 @@ class DialogManageFiles(QtWidgets.QDialog):\napp = None\ntext_dialog = Non

In [36]:
!pip install chromadb

In [34]:
similar_diff = []
for i, data in tqdm(enumerate(test_diff_data[:100]), total=len(test_diff_data[:100]), desc="Processing documents"):
    documents = splitters[language_dict[test_data[i]['lang']]].split_documents([data])
    candidate_id = similarity_search(documents)
    if candidate_id != -1:
        similar_diff.append(diff_data[candidate_id-1])
    else:
        similar_diff.append(None)

data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data[:100]):
    if sim_diff is not None:
        item = {
            'sim_msg': msg_data[sim_diff.metadata['seq_num']],
            'sim_diff': sim_diff.page_content,
            'org_diff': test_diff.page_content
        }
        data.append(item)
    else:
        item = {
            'sim_msg': None,
            'sim_diff': None,
            'org_diff': test_diff.page_content
        }
        data.append(item)

# Write the data to a JSON file
with open('../data/chronicle/rag_dev/codebert/data_with_similar_diff_codebert_split_query.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

Processing documents:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='diff --git a/embark-ui/src/components/TextEditor.js b/embark-ui/src/components/TextEditor.js @@ -120,8 +120,8 @@ class TextEditor extends React.Component {\n}\ncomponentDidUpdate(prevProps) {\n- if (this.props.currentFile.content && this.props.currentFile.content !== prevProps.currentFile.content) {', metadata={'seq_num': 475314, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\chronicle\\chronicle_rag_db.json'}), -0.8607339656495165), (Document(page_content='diff --git a/packages/idyll-components/src/desmos.js b/packages/idyll-components/src/desmos.js @@ -12,6 +12,14 @@ class Desmos extends React.Component {\n? this.generateId(this.props.id)\n: `desmos-${desmosGraphCount++}`\n};\n+ if (\n+ (!this.props.apiKey || this.props.apiKey === desmosApiKey) &&', metadata={'seq_num': 478644, 'source': 'D:\\T

In [7]:
retriever = db.as_retriever()

In [35]:
retriever_2 = db_2.as_retriever()

In [27]:
from tqdm.auto import tqdm

In [55]:
test_diff_data[1].page_content

"diff --git a/src/components/FilterCollapse.js b/src/components/FilterCollapse.js @@ -284,7 +284,7 @@ class FilterCollapse extends Component {\n}\ncomponentDidUpdate(prevProps, prevState) {\n- if (prevProps.filter === this.props.filter) {\n+ if (prevProps.filter !== this.props.filter) {\nQuery.set('f', this.state.filterInput);\n}\n}\n"

In [8]:
retriever.get_relevant_documents(test_diff_data[0].page_content)[0]

Document(page_content='diff --git a/fedn/fedn/common/tracer/mongotracer.py b/fedn/fedn/common/tracer/mongotracer.py @@ -37,7 +37,7 @@ class MongoTracer(Tracer):\nprint("LOG: \\n {} \\n".format(data), flush=True)\n- if self.status:\n+ if self.status!=None:\nself.status.insert_one(data)\ndef drop_round_time(self):', metadata={'seq_num': 166956, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\chronicle\\chronicle_rag_db.json'})

In [36]:
retriever_2.get_relevant_documents(test_diff_data[0].page_content)[0]

Document(page_content="diff --git a/src/components/dashboard/Dashboard.js b/src/components/dashboard/Dashboard.js @@ -66,28 +66,35 @@ class Dashboard extends Component<DashboardProps, DashboardState> {\ngetInitialFeed(this.props.store)\n}\n- handleFeedSelection = (receipt, horizontal) => {\n- this.setState({ horizontal })\n- }\n-\n- showNewFeedEvent = async event => {\n- const item = await userStorage.getStandardizedFeedByTransactionHash(event)\n- log.info('ITEM...', { item })\n- if (item) {\n+ showEventModal = item => {\nthis.setState({\ncurrentFeedProps: {\nitem,\nstyles: {\nflex: 1,\nalignSelf: 'flex-start',\n- height: '90vh',\n+ height: '100vh',\nposition: 'absolute',\nwidth: '100%',\n- padding: normalize(10)\n+ paddingTop: normalize(30),\n+ paddingBottom: normalize(30),\n+ paddingLeft: normalize(10),\n+ paddingRight: normalize(10),\n+ backgroundColor: 'rgba(0, 0, 0, 0.7)'\n},\nonPress: this.closeFeedEvent\n}\n})\n+ }\n+\n+ handleFeedSelection = (receipt, horizontal) => {\n+ this.s

In [56]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data[:100]), total=len(test_diff_data[:100]), desc="Processing documents"):
    query = test_data.page_content
    similar_diff.append(diff_data[retriever.get_relevant_documents(query)[0].metadata['seq_num']-1])

data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data[:100]):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff_codebert.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

Processing documents:   0%|          | 0/100 [00:00<?, ?it/s]

In [57]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data[:100]), total=len(test_diff_data[:100]), desc="Processing documents"):
    query = test_data.page_content
    similar_diff.append(diff_data[retriever_2.get_relevant_documents(query)[0].metadata['seq_num']-1])

data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data[:100]):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff_mxbai.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

Processing documents:   0%|          | 0/100 [00:00<?, ?it/s]

In [48]:
data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

In [49]:
# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff_codebert.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
query =test_diff_data[0].page_content
documents = splitters[0].create_documents([query])

In [ ]:
docs = retriever.get_relevant_documents(documents[0].page_content)

In [ ]:
db.similarity_search_with_relevance_scores(documents[0].page_content)[1]

(Document(page_content='diff --git a/MPChartLib/src/main/java/com/github/mikephil/charting/renderer/BubbleChartRenderer.java \nppp b/MPChartLib/src/main/java/com/github/mikephil/charting/renderer/BubbleChartRenderer.java \nprotected void drawDataSet(Canvas c,IBubbleDataSet dataSet){ \nif(! mViewPortHandler.isInBoundsRight(pointBuffer[0]- shapeHalf)) \nbreak;- final int color=dataSet.getColor(( int)entry.getX());+final int color=dataSet.getColor(j); \nmRenderPaint.setColor(color); \nc.drawCircle(pointBuffer[0], pointBuffer[1], shapeHalf,mRenderPaint); \n', metadata={'seq_num': 27766, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\rag_data.json'}),
 0.736459085862412)

Processing documents: 100%|██████████| 5000/5000 [1:46:31<00:00,  1.28s/it]  


In [ ]:
with open('../data/msg_nngen_nmt_codebert_chatgpt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/msg_nngen_nmt_codebert_chatgpt_rag.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [ ]:
len(gpt_msg)

5000